In [68]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from math import sqrt

In [2]:
data = pd.read_csv('2017-2018_data_joined_3hforecast.csv')
#data.columns = ['Id','Text', 'Label']

#data = data.drop("id", axis=1)
data.head()

,timestamp,MpKrakAlKras-PM10-1g,Przewidywania1,Przewidywania2,Przewidywania3,Weather,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar)
0,2017-01-01 01:00:00,213.733,213.868,221.923,244.690,Ice fog.,0.0,-6.0,1.0,0.93,10.24
1,2017-01-01 02:00:00,213.868,221.923,244.690,224.673,Ice fog.,190.0,-7.0,2.0,0.93,10.24
2,2017-01-01 03:00:00,221.923,244.690,224.673,212.159,Ice fog.,200.0,-4.0,3.0,0.80,10.23
3,2017-01-01 04:00:00,244.690,224.673,212.159,198.375,Ice fog.,210.0,-7.0,3.0,0.86,10.22
4,2017-01-01 05:00:00,224.673,212.159,198.375,191.881,Ice fog.,200.0,-6.0,3.0,0.80,10.22


In [62]:
data.Weather.unique()
weather_dict = {}
weathers = data.Weather.unique()

for num, name in enumerate(weathers, start=1):
    weather_dict[name] = num
weather_dict

#data["weather_id"] = data["Weather"].apply(lambda x:weather_dict[x])
#weather_dict[rows.Weather]
#data.head(30)

{'Ice fog.': 1,
 'Clear.': 2,
 'Sunny.': 3,
 'Haze.': 4,
 'Partly sunny.': 5,
 'Light snow. Ice fog.': 6,
 'Snow. Ice fog.': 7,
 'Light snow. Passing clouds.': 8,
 'Light snow. Partly cloudy.': 9,
 'Partly cloudy.': 10,
 'Passing clouds.': 11,
 'Broken clouds.': 12,
 'Snow flurries. Broken clouds.': 13,
 'Snow. Partly cloudy.': 14,
 'Snow. Passing clouds.': 15,
 'Heavy snow. Partly cloudy.': 16,
 'Snow. Partly sunny.': 17,
 'Light snow. Partly sunny.': 18,
 'Scattered clouds.': 19,
 'Snow flurries. Partly cloudy.': 20,
 'Snow flurries. Ice fog.': 21,
 'Heavy snow. Ice fog.': 22,
 'Snow flurries. Passing clouds.': 23,
 'Light snow. Broken clouds.': 24,
 'Snow flurries. Partly sunny.': 25,
 'Light snow. Overcast.': 26,
 'Light snow. Mostly cloudy.': 27,
 'Mostly cloudy.': 28,
 'Overcast.': 29,
 'Low clouds.': 30,
 'Light snow. Low clouds.': 31,
 'More clouds than sun.': 32,
 'Fog.': 33,
 'Light snow. Fog.': 34,
 'Light freezing rain. Ice fog.': 35,
 'Icy mix. Ice fog.': 36,
 'Light rain.

In [51]:
data["hour"] = data["timestamp"].apply(lambda x:x[-8:-6]) #pora dnia chyba ma znaczenie (godziny szczytu itd)
data["month"] = data["timestamp"].apply(lambda x:x[5:7]) #pora roku też może mieć wpływ

data_useful = data[["hour","month","MpKrakAlKras-PM10-1g","Direction(deg)","Temp(C)","Wind(m/s)","Humidity","Barometer(mbar)", "Przewidywania1","Przewidywania2","Przewidywania3"]]

In [57]:
#[TRAIN] feature scaling

scaler = StandardScaler()
#data_train_mean = data_useful.mean()
#data_train_std = data_useful.std(axis=0)
data_scale = pd.DataFrame(scaler.fit_transform(data_useful),columns=data_useful.columns)
data_scale

,hour,month,MpKrakAlKras-PM10-1g,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar),Przewidywania1,Przewidywania2,Przewidywania3
0,-1.516289,-1.595378,3.404577,-1.230647,-1.640521,-1.070446,0.921079,1.008337,3.409070,3.584589,4.078041
1,-1.371791,-1.595378,3.407489,0.460061,-1.746218,-0.628150,0.921079,1.008337,3.582907,4.076108,3.645717
2,-1.227292,-1.595378,3.581263,0.549046,-1.429128,-0.185855,0.163394,0.880454,4.074247,3.643959,3.375442
3,-1.082794,-1.595378,4.072425,0.638031,-1.746218,-0.185855,0.513095,0.752572,3.642256,3.373793,3.077738
4,-0.938295,-1.595378,3.640590,0.549046,-1.640521,-0.185855,0.163394,0.752572,3.372188,3.076209,2.937482
...,...,...,...,...,...,...,...,...,...,...,...
17277,0.795686,1.582806,-0.525243,1.438892,-0.794950,0.256440,0.221677,1.519869,-0.527653,-0.509954,-0.301926
17278,0.940185,1.582806,-0.527810,1.349908,-0.794950,-0.628150,0.221677,1.647752,-0.510142,-0.302094,-0.054448
17279,1.084683,1.582806,-0.510306,1.082954,-0.794950,-0.185855,0.571378,1.647752,-0.302358,-0.054716,-0.110827
17280,1.229182,1.582806,-0.302597,1.260923,-0.794950,0.698735,0.571378,1.647752,-0.055069,-0.111072,0.013747


In [58]:
#weather_id trzeba by dodać jako one-hot a nie słownik, tylko że wtedy mamy 100 dodatkowych kolumn

X = data_scale[["hour","month","MpKrakAlKras-PM10-1g","Direction(deg)","Temp(C)","Wind(m/s)","Humidity","Barometer(mbar)"]]

y = data_scale[["Przewidywania1","Przewidywania2","Przewidywania3"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=0)

In [ ]:
X.head(10)
y.head(10)

# 2

In [64]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

In [65]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
15553/15553 [==============================] - 2s 130us/step - loss: -142532877.9106 - accuracy: 0.0000e+00
Epoch 2/100
15553/15553 [==============================] - 2s 142us/step - loss: -146632638.1481 - accuracy: 0.0000e+00
Epoch 3/100
15553/15553 [==============================] - 3s 172us/step - loss: -150793986.7814 - accuracy: 0.0000e+00
Epoch 4/100
15553/15553 [==============================] - 2s 149us/step - loss: -155019419.6678 - accuracy: 0.0000e+00
Epoch 5/100
15553/15553 [==============================] - 4s 231us/step - loss: -159333931.1388 - accuracy: 0.0000e+00
Epoch 6/100
15553/15553 [==============================] - 3s 205us/step - loss: -163741460.3732 - accuracy: 0.0000e+00
Epoch 7/100
15553/15553 [==============================] - 3s 177us/step - loss: -168230480.8771 - accuracy: 0.0000e+00
Epoch 8/100
15553/15553 [==============================] - 2s 121us/step - loss: -172786886.2565 - accuracy: 0.0000e+00 1s - loss: -17
Epoch 9/100
15553/15553 [

15553/15553 [==============================] - 2s 150us/step - loss: -618866150.7922 - accuracy: 0.0000e+00
Epoch 69/100
15553/15553 [==============================] - 2s 141us/step - loss: -629676336.9274 - accuracy: 0.0000e+00
Epoch 70/100
15553/15553 [==============================] - 2s 126us/step - loss: -640630808.9617 - accuracy: 0.0000e+00
Epoch 71/100
15553/15553 [==============================] - 2s 156us/step - loss: -651681648.4492 - accuracy: 0.0000e+00
Epoch 72/100
15553/15553 [==============================] - 2s 130us/step - loss: -662827166.3072 - accuracy: 0.0000e+00
Epoch 73/100
15553/15553 [==============================] - 2s 137us/step - loss: -674073331.1649 - accuracy: 0.0000e+00
Epoch 74/100
15553/15553 [==============================] - 2s 115us/step - loss: -685473528.8222 - accuracy: 0.0000e+00
Epoch 75/100
15553/15553 [==============================] - 3s 201us/step - loss: -696983363.8309 - accuracy: 0.0000e+00
Epoch 76/100
15553/15553 [===================

In [70]:
y_pred = model.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
rms

0.8595896366774926